# Recommender System

### Neighborhood-based model using Term Frequency-Inverse Document Frequency (TF-IDF) and Cosine Similarity

In this notebook, a neighborhood-based model is implemented to generate recommendations for a target user, by using cosine similarities on a TF-IDF weighted utility matrix.

We are using the library [`implicit`](https://github.com/benfred/implicit) to calculates tfidf_weight, [`scipy`](https://www.scipy.org) to implement sparse matrix, and [`sklearn`](http://scikit-learn.org/stable/) to calculate cosine similarity.

The following steps are conducted: 
- Utlity Matrix is generated, where every entry represents a Term Frequency(TF).
- Based on the utility matrix, a TF-IDF weight matrix is calculated.
- Given a target user (a row in the TF-IDF matrix), the cosine similarity vector of the target user is produced.
- Select top K similar users and use PriorityQueue to select top N products based on overall sales in the candidate recommendations.

In [1]:
### Imports

from implicit.nearest_neighbours import tfidf_weight
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
from pathlib import Path
from numpy import bincount, log, sqrt

import scipy.sparse as sparse
import implicit
import pandas as pd
import numpy as np
import pickle
import time
import heapq

In [2]:
### Helper Functions

def sparsity(matrix):
    """
    Given a matrix, returns its sparsity
    """
    total_size = matrix.shape[0] * matrix.shape[1]
    actual_size = matrix.size
    sparsity = (1 - (actual_size / total_size)) * 100
    return(sparsity)


def get_k_popular(k, df_merged_order_products_prior):
    """
    Returns the `k` most popular products based on purchase count in the dataset
    """
    popular_products = list(df_merged_order_products_prior["product_id"].value_counts().head(k).index)
    return popular_products


def make_prior_data():
    """
    Generates the prior dataset including prior_user_products and product_frequency
    """
    # Read prior order csv
    df_order_products_prior = pd.read_csv("../data/order_products__prior.csv")
    current_order_user_df = df_orders.loc[(df_orders.eval_set == "prior")].reset_index()
    current_order_user_df = current_order_user_df[["order_id", "user_id"]]

    assert len(current_order_user_df["order_id"].unique()) == len(df_order_products_prior["order_id"].unique())

    # Group product_id for each order into products
    df_order_products_prior = df_order_products_prior[["order_id", "product_id"]]
    df_product_frequency = df_order_products_prior['product_id'].value_counts()
    df_order_products_prior = df_order_products_prior.groupby("order_id")["product_id"].apply(list).reset_index().rename(columns={"product_id": "products"})
    
    
    assert current_order_user_df.size == df_order_products_prior.size

    df_prior_user_products = pd.merge(current_order_user_df, df_order_products_prior, on="order_id")
    df_prior_user_products = df_prior_user_products[["user_id", "products"]]
    df_prior_user_products = df_prior_user_products.groupby("user_id")["products"].agg(sum).reset_index()

    return df_prior_user_products, df_product_frequency


def make_test_data(test_data_path, df_orders, df_order_products_train):
    """
    Generates the test dataset and saves it to disk at the given path
    """
    start = time.time()
    print("Creating test data ...")

    # Read train csv
    df_order_user_current = df_orders.loc[(df_orders.eval_set == "train")].reset_index()
    df_order_user_current = df_order_user_current[["order_id", "user_id"]]
    
    # Sanity check #1: `current_order_user_df` and `df_order_products_train` should have the same number of 
    # unique order ids
    assert len(df_order_user_current["order_id"].unique()) == len(df_order_products_train["order_id"].unique())

    # Convert train dataframe to a similar format
    df_order_products_test = df_order_products_train[["order_id", "product_id"]]
    df_order_products_test = df_order_products_test.groupby("order_id")["product_id"].apply(list).reset_index().rename(columns={"product_id": "products"})

    # Sanity check #2: `df_order_products_test` and `df_order_user_current` should have the same number of 
    # records before attempting to merge them
    assert df_order_products_test.size == df_order_user_current.size

    # Merge on order id
    df_user_products_test = pd.merge(df_order_user_current, df_order_products_test, on="order_id")
    df_user_products_test = df_user_products_test[["user_id", "products"]]

    # Write to disk
    df_user_products_test.to_csv(test_data_path, index_label=False, index=False)
    
    print("Completed in {:.2f}s".format(time.time() - start))


def save_data_to_disk(dataframe, df_name):
    """
    Save the data to disk
    """
    filepath = "../data/df_{}.pkl".format(df_name)
    dataframe.to_pickle(filepath)

# Load datasets

In [3]:
# Order datasets
df_order_products_prior = pd.read_csv("../data/order_products__prior.csv")
df_order_products_train = pd.read_csv("../data/order_products__train.csv")
df_orders = pd.read_csv("../data/orders.csv") 

# Products
df_products = pd.read_csv("../data/products.csv")

# Merge prior orders and products
df_merged_order_products_prior = pd.merge(df_order_products_prior, df_products, on="product_id", how="left")

rec_items = 10
sim_users = 5

In [4]:
# Skip this block if you already have the df_user_products.pkl and df_product_frequency.pkl in the disk
# Make prior data
# Running time: 3 min
df_prior_user_products, df_product_frequency = make_prior_data()

# save data to disk, running time : 2 mi
save_data_to_disk(df_prior_user_products, "user_products")
save_data_to_disk(df_product_frequency, "product_frequency")

In [5]:
# Read user_products and product_frequency from the disk
df_prior_user_products = pd.read_pickle("../data/df_user_products.pkl")
df_product_frequency = pd.read_pickle("../data/df_product_frequency.pkl")
df_product_frequency = pd.DataFrame(df_product_frequency).rename(columns={"product_id": "frequency"})

In [6]:
# Make test data
REBUILD_TEST_DATA = True
test_data_path = "../data/user_products__test.csv"
if REBUILD_TEST_DATA or not Path(test_data_path).is_file():
    make_test_data(test_data_path, df_orders, df_order_products_train)
df_user_products_test = pd.read_csv(test_data_path)

Creating test data ...
Completed in 4.25s


In [7]:
df_user_products_test.head()

,user_id,products
0,1,[1]
1,3,[9]
2,5,[20]
3,6,[26]
4,7,[35]


# Load Product Item Matrix

In [8]:
def get_user_product_prior_df(filepath, df_orders, df_order_products_prior):
    """
    Generates a dataframe of users and their prior products purchases, and writes it to disk at the given path
    """
    start = time.time()
    print("Creating prior user product data frame ...")
    
    # Consider ony "prior" orders and remove all columns except `user_id` from `df_orders`
    df_order_user_prior = df_orders.loc[df_orders.eval_set == "prior"]
    df_order_user_prior = df_order_user_prior[["order_id", "user_id"]]
    
    # Remove all columns except order_id and user_id from df_orders and 
    # merge the above on `order_id` and remove `order_id`
    df_merged = pd.merge(df_order_user_prior, df_order_products_prior[["order_id", "product_id"]], on="order_id")
    df_user_product_prior = df_merged[["user_id", "product_id"]]
    df_user_product_prior = df_user_product_prior.groupby(["user_id", "product_id"]).size().reset_index().rename(columns={0:"quantity"})
    
    # Write to disk
    df_user_product_prior.to_csv(filepath, index_label=False, index=False)

    print("Completed in {:.2f}s".format(time.time() - start))


# Build dataframe of users, products and quantity bought using prior datasets
REBUILD_MATRIX_DF = True
matrix_df_path = "../data/user_products__prior.csv"
if REBUILD_MATRIX_DF or not Path(matrix_df_path).is_file():
    get_user_product_prior_df(matrix_df_path, df_orders, df_order_products_prior)
df_user_product_prior = pd.read_csv(matrix_df_path)
df_user_product_prior["user_id"] = df_user_product_prior["user_id"].astype("category")
df_user_product_prior["product_id"] = df_user_product_prior["product_id"].astype("category")

Creating prior user product data frame ...
Completed in 2.38s


In [9]:
def build_product_user_matrix(matrix_path, df_user_product_prior):
    """
    Generates a utility matrix representing purchase history of users, and writes it to disk.
    Rows and Columns represent products and users respectively.
    """
    start = time.time()
    print("Creating product user matrix ...")
    
    # Make the dataframe a sparse matrix
    df_user_product_prior["user_id"] = df_user_product_prior["user_id"].astype("category")
    df_user_product_prior["product_id"] = df_user_product_prior["product_id"].astype("category")
    product_user_matrix = sparse.coo_matrix((df_user_product_prior["quantity"],
                                            (df_user_product_prior["product_id"].cat.codes.copy(),
                                             df_user_product_prior["user_id"].cat.codes.copy())))
    
    sparse.save_npz(matrix_path, product_user_matrix)
    
    print("Completed in {:.2f}s".format(time.time() - start))


# Get the `product x user` matrix
REBUILD_MATRIX = True
matrix_path = "../data/product_user_matrix.npz"
if REBUILD_MATRIX or not Path(matrix_path).is_file():
    build_product_user_matrix(matrix_path, df_user_product_prior)
product_user_matrix = sparse.load_npz(matrix_path).tocsr()

Creating product user matrix ...
Completed in 0.28s


In [10]:
sparsity(product_user_matrix)

99.99555128671498

# Term Frequency-Inverse Document Frequency

In [11]:
# Fetch Term Frequency matrix
user_product_matrix = product_user_matrix.T

In [12]:
def tfidf_weight(tf):
    """
    Given a Term Frequency matrix
    Returns a TF-IDF weight matrix
    """
    
    tf_idf = coo_matrix(tf)

    # calculate IDF
    N = float(tf_idf.shape[0])
    idf = log(N / (1 + bincount(tf_idf.col)))

    # apply TF-IDF adjustment
    tf_idf.data = sqrt(tf_idf.data) * idf[tf_idf.col]
    return tf_idf

tf_idf = tfidf_weight(user_product_matrix)

# convert to Compressed Sparse Row format
tf_idf = tf_idf.tocsr()

## Example Recommendation

In [13]:
def generateRecommendations(target_user, cos_vec, K, N):
    """
    Given a target_user (a row), a cosine similarity vector, the number of similar users K, 
          the number of products to be recommended.
    Returns product set by target user and N recommendations
    """
    
    # Select top K similar users
    top_K_similar_users = heapq.nlargest(K+1, range(len(cos_vec)), cos_vec.take)

    # Initialize the result for recommendations
    recommendations = []
    
    # Exclude the user with same purchase history (1.00000) as the target user and implement set-minus
    products_target_user = df_prior_user_products.loc[df_prior_user_products['user_id'] == target_user_id].products

    # Products of Target User
    productset_target_user = set(products_target_user.tolist()[0])

    # Fetch the preliminary recommendations
    for similar_user_id in top_K_similar_users:
        
        products_similar_user = df_prior_user_products.loc[df_prior_user_products['user_id'] == similar_user_id + 1].products

        # Recommend the products bought by the user who firstly differs in the purchase history from A.
        candidate_recommendation = set(products_similar_user.tolist()[0]) - productset_target_user

        # If similar_user_id equals to target_user_id or the candidate_recommendation is empty,
        # skip current user
        if similar_user_id == target_user_id or not candidate_recommendation: continue

        # One candidate_recommendation found, and extend it to the result
        recommendations.extend(candidate_recommendation)

        # If length of recommendations exceed N, break
        # Needed because this will ensure the recommentations are the products bought by most similar users
        if len(recommendations) > N: break
        
    # Pick the top N popularity (overall sales) to recommend
    h = []
    for rec in recommendations:
        heapq.heappush(h, (df_product_frequency.loc[rec]['frequency'], rec))
        if len(h) > N:
            heapq.heappop(h)
            
    return productset_target_user, [item[1] for item in h]

In [14]:
users = pd.read_csv("../Data/users.csv")

col = ['model','reviewerID','prior_products','rec_products']
model='tfidf'
tfidf_rec_items = pd.DataFrame(columns=col)

for r, value in users.iterrows():
    
    target_user_id = value[0]
    
    # Fetch row of target user
    target_user = tf_idf[target_user_id - 1]

    # Calculate Cosine Similarity Vector of target user
    similarities = cosine_similarity(tf_idf, target_user, False)

    productset_target_user, recommendations = generateRecommendations(target_user, similarities.toarray(), sim_users, rec_items)
    
    print('Actual products bought by User {}:'.format(target_user_id))
    print(productset_target_user)
    print()
    print('Recommended products for User {}:'.format(target_user_id))
    print(recommendations)
    
    tfidf_rec_items.loc[len(tfidf_rec_items)] = [model,target_user_id,productset_target_user,recommendations]

Actual products bought by User 34418:
{6025, 10159, 113712, 1944, 2079}

Recommended products for User 34418:
[37322, 33589, 9996, 33587, 15391, 9103, 5845, 971, 55189, 1624]
Actual products bought by User 41538:
{24165, 92718, 25519, 130069, 130070}

Recommended products for User 41538:
[14026, 24949, 72347, 76071, 75182, 29744, 7090, 3831, 23003, 1658]
Actual products bought by User 8132:
{7556, 11929, 6431, 1695, 9127, 31274, 22578, 3897, 36928, 36929, 36930, 36931, 36932, 36934, 36935, 32206, 8538, 15835, 3688, 31336, 26232}

Recommended products for User 8132:
[127513, 47002, 928, 7664, 8504, 13805, 8138, 12292, 10593, 7867]
Actual products bought by User 36218:
{30080, 6506, 2737, 11091, 601}

Recommended products for User 36218:
[2740, 5442, 137895, 19992, 18540, 5491, 2739, 193, 194, 14724]
Actual products bought by User 9127:
{40609, 40610, 40611, 54276, 11589, 35180, 40174, 29744, 36978, 19956, 22932, 724, 1375}

Recommended products for User 9127:
[107910, 155001, 17877, 157

In [15]:
tfidf_rec_items.to_csv('../data/tfidf_rec_items.csv', index=False)

In [16]:
# # Selecting one user to test
# target_user_id = 1

# # Fetch row of target user
# target_user = tf_idf[target_user_id - 1]

# # Calculate Cosine Similarity Vector of target user
# similarities = cosine_similarity(tf_idf, target_user, False)

# productset_target_user, recommendations = generateRecommendations(target_user, similarities.toarray(), sim_users, rec_items)

In [17]:
# # Output the product_name of Target User's products as well as Recommendations
# print('Actual products bought by User {}:'.format(target_user_id))
# print(productset_target_user)
# print()
# print('Recommended products for User {}:'.format(target_user_id))
# print(recommendations)

# Evaluation

In [18]:
# Get the 10 most popular products
popular_products = get_k_popular(rec_items, df_merged_order_products_prior)

In [19]:
def recall_score(actual, pred):
    """
    Given two lists representing actual and predicted values
    Returns the recall of the prediction
    """
    if len(actual) == 0:
        return 0
    actual, pred = set(actual), set(pred)
    return len(actual.intersection(pred)) / len(actual)

def new_products(row):
    """
    Given a row in the test dataset
    Returns the list of new products purchased
    """
    actual = row["products"][1:-1]
    actual = set([int(p.strip()) for p in actual.strip().split(",")])
    products_target_user = df_prior_user_products.loc[df_prior_user_products['user_id'] == row["user_id"]].products
    liked = set(products_target_user.tolist()[0])
    return actual - liked

def popular_recommend(row):
    """
    Given a row in the test dataset
    Returns the recall score when popular products are recommended
    """
    actual = new_products(row)
    return recall_score(actual, popular_products)

def tfidf_recommend(row):
    """
    Given a row in the test dataset
    Returns the recall score when our model recommends products
    """
    actual = row["products"][1:-1]
    actual = [int(p.strip()) for p in actual.strip().split(",")]
    target_user = tf_idf[row["user_id"] - 1]
    similarities = cosine_similarity(tf_idf, target_user, False)
    cos_vec = similarities.toarray()
    productset_target_user, recommended = generateRecommendations(target_user, cos_vec, sim_users, rec_items)

    cur_recall_score = recall_score(actual, recommended)
    
    global count, progress, recall_sum
    count += 1; recall_sum += cur_recall_score
    if level[progress] and int(count / total * 10) - 1 == progress:
        level[progress] = False
        progress += 1
        print("{:.1f}% completed, current mean of recall = {}".format(progress * 10, recall_sum / count))    
    
    return cur_recall_score

def build_eval_df(filepath, df_user_products_test, subset=None):
    """
    Builds a dataframe of recall values of the baseline and our model for all the users
    in the test data, and saves its to disk at `filepath`
    """
    start = time.time()
    print("Building dataframe with recall values ...")
    
    df_eval = df_user_products_test.copy()
    if subset:
        df_eval = df_eval.sample(n=int(len(df_eval) * subset), random_state=7)
    df_eval["popular_score"] = df_eval.apply(popular_recommend, axis=1)
    df_eval["tfidf_score"] = df_eval.apply(tfidf_recommend, axis=1)
    df_eval.to_csv(filepath, index=False) #, index_label=False)
    
    print("Completed in {:.2f}s".format(time.time() - start))    


# Get the dataframe with recall values of the baseline and the model
REBUILD_EVAL_DF = True
subset = 0.2

# How many users in the test?
total = len(df_user_products_test) * subset

# Counter
count = 0
progress = 0
recall_sum = 0
level = [True] * 10

# Estimated 3 hours to run 20% of the test dataset
eval_path = "../data/eval/eval_tfidf_{}.csv".format(subset if subset is not None else "full")
if REBUILD_EVAL_DF or not Path(eval_path).exists():
    build_eval_df(eval_path, df_user_products_test, subset=subset)
df_eval = pd.read_csv(eval_path)

Building dataframe with recall values ...
10.0% completed, current mean of recall = 0.002789400278940028
20.0% completed, current mean of recall = 0.0027913468248429866
30.0% completed, current mean of recall = 0.0037226617031177293
40.0% completed, current mean of recall = 0.004188481675392671
50.0% completed, current mean of recall = 0.0036302708740575257
60.0% completed, current mean of recall = 0.0039562485454968585
70.0% completed, current mean of recall = 0.003989626969878316
80.0% completed, current mean of recall = 0.0038401117123407227
90.0% completed, current mean of recall = 0.003568657874321179
Completed in 446.47s


In [20]:
# Mean recall scores
model_mean_recall, baseline_mean_recall = np.mean(df_eval["tfidf_score"]), np.mean(df_eval["popular_score"])
print("Model: {:.2f}%".format(model_mean_recall * 100))
print("Baseline: {:.2f}%".format(baseline_mean_recall * 100))

Model: 0.41%
Baseline: 0.66%
